In [1]:
import os
import pickle
import random
import sys
import time
import math
import itertools
from pathlib import Path

import polars as pl
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import potsimloader as psl
import models as mdl
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.multiprocessing as mp
from torchinfo import summary
from torchmetrics.regression import R2Score

In [2]:
TARGET_FEATURES = {
    "NTotL1": ["DayAfterPlant", "NApp", "Rain", "SolarRad", "AirTempC"],
    "NTotL2": ["DayAfterPlant", "NApp", "Rain", "SolarRad", "AirTempC"],
    "SWatL1": ['DayAfterPlant','IrrgThresh', 'NLeach', 'Irrg', 'Rain'],
    "SWatL2": ['DayAfterPlant','IrrgThresh', 'NLeach', 'Irrg', 'Rain'],
    "NLeach": ['NTotL1', 'NTotL2', 'Rain', 'SolarRad', 'AirTempC'],
    "NPlantUp": ['DayAfterPlant', 'NTotL1', 'NTotL2', 'Rain', 'SolarRad', 'AirTempC']
}

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)  # Set seed for PyTorch CPU
    torch.cuda.manual_seed(seed)  # Set seed for PyTorch GPU (if using CUDA)
    np.random.seed(seed)  # Set seed for NumPy
    random.seed(seed)  # Set seed for Python's random module
set_seed(42)

In [7]:
from utils import preprocessing as ppsr


In [ ]:
def generate_treatments(n_values):
    combis = list(itertools.product(n_values, repeat=3))
    return ['-'.join(map(str, t)) for t in combis if sum(t) <= 400]

dssat_file = "data/potsim_yearly/potsim_2001.parquet"
weather_file = "data/weather.parquet"

n_values = [0, 56, 112, 168, 196]
# n_values = [28, 84, 140]
treatments = generate_treatments(n_values)

scenarios = {
    "Year": list(range(2004, 2024)),
    "Treatment": treatments,
    "PlantingDay": [1, 15, 29, 43, 57],
    # "IrrgDep": [30, 50],
    # "IrrgThresh": [50, 70, 90]
}
# scenarios = {
#     "Year": list(range(2018, 2024)),
#     "Treatment": treatments,
#     "PlantingDay": [8, 22, 36, 50, 64],
#     "IrrgDep": [40, 60],
#     "IrrgThresh": [60, 80, 100]
# }
usecols = ['Year', 'Date', 'Treatment', 'NFirstApp','PlantingDay', 'IrrgDep',
           'IrrgThresh', 'DayAfterPlant', 'NApp', 'NLeach', 'NPlantUp', 'NTotL1', 
           'NTotL2', 'Irrg', 'SWatL1', 'SWatL2', 'Rain', 'SolarRad', 'AirTempC']
    
mask = (
    ((pl.col("NFirstApp") == "Npl") & (pl.col("DayAfterPlant") >= -1)) |
    ((pl.col("NFirstApp") == "Npre") & (pl.col("DayAfterPlant") >= -37))
)

data = psl.read_data(
    dataset_path = dssat_file,
    weather_path = weather_file,
    usecols = usecols,
    scenarios=scenarios,
    lazy = False
)
df = data.filter(mask).to_pandas()
numcols = df.select_dtypes(exclude=['category']).columns
df[numcols] = df[numcols].fillna(0)

In [ ]:

x,y= ppsr.process_data()

TypeError: process_data() missing 4 required positional arguments: 'df', 'feats', 'tgt', and 'scaler_path'

In [4]:
def prepare_sequences(df, feature_cols, target_col, seq_len=None):
    scenario_vars = ['Year', 'PlantingDay', 'Treatment', 'NFirstApp', 'OrgIrrgDep', 'OrgIrrgThresh']
    df_sorted = df.sort_values(scenario_vars + ['OrgDayAfterPlant'])
    data = df_sorted[feature_cols].values
    target = df_sorted[target_col].values
    
    # Empty Vectorized window creation in case no data
    if data.size == 0 or len(data) < seq_len:
        return np.empty((0, seq_len, len(feature_cols))), np.empty(0)
    
    # Create group boundaries using vectorized operations
    group_mask = (df_sorted[scenario_vars] != df_sorted[scenario_vars].shift()).any(axis=1).values
    group_ids = np.cumsum(group_mask)
    
    # Create validity mask for full sequences within groups
    valid = np.zeros(len(data) - seq_len + 1, dtype=bool)
    for i in range(len(valid)):
        valid[i] = group_ids[i] == group_ids[i + seq_len - 1]
    
    # Create windows and transpose dimensions to [samples, seq_len, features]
    X = sliding_window_view(data, (seq_len,), axis=0)[valid].transpose(0, 2, 1)
    y = target[seq_len-1:][valid]
    return X, y


def transform_data(df, feature_cols, target_col, scaler_path, mode="transform"):
    scenario_vars = ['Year', 'PlantingDay', 'Treatment', 'NFirstApp', 'OrgIrrgDep', 'OrgIrrgThresh']  
    df_copy = df.copy()
    df_copy['OrgDayAfterPlant'] = df['DayAfterPlant']
    df_copy['OrgIrrgDep'] = df['IrrgDep']
    df_copy['OrgIrrgThresh'] = df['IrrgThresh']
    df_copy = df_copy.sort_values(by=scenario_vars + ['OrgDayAfterPlant'])
    df_copy["NApp"] = df_copy.groupby(scenario_vars, observed=True)["NApp"].transform('cumsum')
    df_copy, _ = mdl.normalize_columns(
        df_copy, feature_cols + [target_col],
        mode=mode, scaler_path=scaler_path
    )
    return df_copy



def process_data(df, feature_cols, target_col, scaler_path, mode="transform", seq_len=None):
    if not all(col in df.columns for col in feature_cols + [target_col]):
        raise ValueError("Some columns not found in dataframe")
        
    df_copy =  transform_data(df, feature_cols, target_col, scaler_path, mode=mode)
    # For non-sequential data preparation
    if seq_len is None:
        X = df_copy[feature_cols].values
        y = df_copy[target_col].values
        del df_copy
        return torch.FloatTensor(X), torch.FloatTensor(y).view(-1, 1)
    
    # For sequence preparation
    X, y = prepare_sequences(df_copy, feature_cols, target_col, seq_len=seq_len)
    del df_copy
    return torch.FloatTensor(X), torch.FloatTensor(y).view(-1, 1)

In [5]:
def split_train_val_test(df):
    # 1. Define year splits
    train_years = list(range(2004, 2016))  # 12 years
    val_years = list(range(2016, 2020))    # 4 years
    test_years = list(range(2020, 2024))    # 4 years
    
    train_data = df[df["Year"].isin(train_years)]
    val_data = df[df["Year"].isin(val_years)]
    test_data = df[df["Year"].isin(test_years)]

    return train_data, val_data, test_data

def random_sample_train_val_test(df):    
    scenario_vars = ['PlantingDay', 'Treatment', 'NFirstApp', 'IrrgDep', 'IrrgThresh']
    groups = df.groupby(scenario_vars, observed=True, sort=False)
    unique_groups = list(groups.groups.keys())
    np.random.shuffle(unique_groups)
    
    # Shuffle and split 70-15-15
    n = len(unique_groups)
    n_train = int(0.6 * n)
    n_val = int(0.2 * n)
    print(f"Scenarios: train({n_train * 12}), val({n_val * 4}), test({(n - n_train - n_val) * 4})")

    train_set = unique_groups[:n_train]
    val_set = unique_groups[n_train:n_train + n_val]
    test_set = unique_groups[n_train + n_val:]
    
    # Split data between train, val, test fro years to reduce matching
    train_data, val_data, test_data = split_train_val_test(df)
    
    def _filter_scenarios(_data, _dset):
        _dset = pd.DataFrame(_dset, columns=scenario_vars)
        for col in scenario_vars:
            if _data[col].dtype.name == 'category':
                _dset[col] = _dset[col].astype(_data[col].dtype)
        return _data.merge(_dset, on=scenario_vars, how='inner')
    
    return (_filter_scenarios(train_data, train_set), 
            _filter_scenarios(val_data, val_set), 
            _filter_scenarios(test_data, test_set))

In [6]:
def generate_treatments(n_values):
    combis = list(itertools.product(n_values, repeat=3))
    return ['-'.join(map(str, t)) for t in combis if sum(t) <= 400]

dssat_file = "data/dssat_scenarios_latest.parquet"
weather_file = "data/weather_2000-2023.parquet"

n_values = [0, 56, 112, 168, 196]
# n_values = [28, 84, 140]
treatments = generate_treatments(n_values)

scenarios = {
    "Year": list(range(2004, 2024)),
    "Treatment": treatments,
    "PlantingDay": [1, 15, 29, 43, 57],
    # "IrrgDep": [30, 50],
    # "IrrgThresh": [50, 70, 90]
}
# scenarios = {
#     "Year": list(range(2018, 2024)),
#     "Treatment": treatments,
#     "PlantingDay": [8, 22, 36, 50, 64],
#     "IrrgDep": [40, 60],
#     "IrrgThresh": [60, 80, 100]
# }
usecols = ['Year', 'Date', 'Treatment', 'NFirstApp','PlantingDay', 'IrrgDep',
           'IrrgThresh', 'DayAfterPlant', 'NApp', 'NLeach', 'NPlantUp', 'NTotL1', 
           'NTotL2', 'Irrg', 'SWatL1', 'SWatL2', 'Rain', 'SolarRad', 'AirTempC']
    
mask = (
    ((pl.col("NFirstApp") == "Npl") & (pl.col("DayAfterPlant") >= -1)) |
    ((pl.col("NFirstApp") == "Npre") & (pl.col("DayAfterPlant") >= -37))
)

data = psl.read_data(
    dataset_path = dssat_file,
    weather_path = weather_file,
    usecols = usecols,
    scenarios=scenarios,
    lazy = False
)
df = data.filter(mask).to_pandas()
numcols = df.select_dtypes(exclude=['category']).columns
df[numcols] = df[numcols].fillna(0)

FileNotFoundError: File not found: data\dssat_scenarios_latest.parquet

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56246400 entries, 0 to 56246399
Data columns (total 19 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Year           int32         
 1   Date           datetime64[ms]
 2   Treatment      category      
 3   NFirstApp      category      
 4   PlantingDay    int32         
 5   IrrgDep        int32         
 6   IrrgThresh     int32         
 7   DayAfterPlant  int32         
 8   NApp           float32       
 9   NLeach         float32       
 10  NPlantUp       float32       
 11  NTotL1         float32       
 12  NTotL2         float32       
 13  Irrg           float32       
 14  SWatL1         float32       
 15  SWatL2         float32       
 16  Rain           float32       
 17  SolarRad       float32       
 18  AirTempC       float32       
dtypes: category(2), datetime64[ms](1), float32(11), int32(5)
memory usage: 3.9 GB


In [8]:
target_col = "NTotL1"
feature_cols = TARGET_FEATURES[target_col]
seq_len = None
lr = 0.005
b_sz = 2048
max_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# model_name = "EncoderOnlyTransformer"
# model = mdl.EncoderOnlyTransformer(
#         input_dim=len(feature_cols), num_layers=2, nhead=4,
#         d_model=128, max_seq_len=seq_len)
# model_name="CNN1D"
# model = mdl.CNN1D(input_dim=len(feature_cols), hidden_size=128)

# model_name = "MLP"
# model = mdl.MLP(input_dim=len(feature_cols), 
#                        hidden_size=128, 
#                        num_layers=2, 
                       # dropout=0.2)

# model_name= "TCN"
# model = mdl.TCN(input_dim=len(feature_cols), 
#                         num_channels=[128, 64, 32, 16, 8], 
#                         kernel_size=3)

# model_name="Linear_Regression"
# model = mdl.LinearRegression(input_dim=len(feature_cols))

# model_name="LSTM"
# model = mdl.LSTM(input_dim=len(feature_cols), num_layers=2, hidden_size=128)

# model.to(device)
# criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-8)
# ampscaler = torch.amp.GradScaler(device=device)
scaler_path=f"models/notebook/{target_col}_scaler_v3.pkl"
# print(summary(model))

In [10]:
# Strategy-1 (equal proportions)
# train_split, val_split, test_split = split_train_val_test(df)

# Strategy-2 (random sampling)
train_split, val_split, test_split = random_sample_train_val_test(df)

# # Process Data
X_train, y_train = process_data(train_split, feature_cols, target_col, 
                                scaler_path=scaler_path, mode="fit", seq_len=seq_len)
X_val, y_val = process_data(val_split, feature_cols, target_col,
                            scaler_path=scaler_path, mode="transform", seq_len=seq_len)
X_test, y_test = process_data(test_split, feature_cols, target_col,
                              scaler_path=scaler_path, mode="transform", seq_len=seq_len)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

Scenarios: train(160704), val(17856), test(17856)
torch.Size([20267712, 5]) torch.Size([20267712, 1]) torch.Size([2246256, 5]) torch.Size([2246256, 1]) torch.Size([2247120, 5]) torch.Size([2247120, 1])


In [14]:
inverse_normalize(y_train[:10], target_col, scaler_path)

array([[9.23],
       [9.3 ],
       [9.37],
       [9.44],
       [9.51],
       [9.57],
       [9.63],
       [9.69],
       [9.75],
       [9.81]], dtype=float32)

In [16]:
train_loader = DataLoader(TensorDataset(X_train, y_train),
                          batch_size=b_sz, shuffle=True, 
                          num_workers=10, persistent_workers=True, 
                          prefetch_factor=4, pin_memory=True)
val_loader = DataLoader(TensorDataset(X_val, y_val),
                          batch_size=b_sz, shuffle=False, 
                          num_workers=4, persistent_workers=True, 
                          prefetch_factor=4, pin_memory=True)

In [13]:
def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = root_mean_squared_error(y_true, y_pred) 
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

def inverse_normalize(arr, target_col, scaler_path):
        with open(scaler_path, 'rb') as f:
            params = pickle.load(f)
        if target_col not in params:
            raise ValueError(f'{taregt_col} not present in scaler file.')
        min_val = params[target_col]['min']
        max_val = params[target_col]['max']
        return (np.array(arr) * (max_val - min_val)) + min_val
    
def get_scaler_min_max(target_col, scaler_path, device):
        with open(scaler_path, 'rb') as f:
            params = pickle.load(f)
        if target_col not in params:
            raise ValueError(f'{taregt_col} not present in scaler file.')
        min_val = torch.tensor(params[target_col]['min'], device=device)
        max_val = torch.tensor(params[target_col]['max'], device=device)
        return min_val, max_val

In [18]:
def train_epoch(model, loader, optimizer, 
                criterion, device, 
                min_val, max_val,
                ampscaler=None):
    model.train()
    total_loss = 0.0
    r2_metric = R2Score().to(device)
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        if ampscaler:
            with torch.amp.autocast(device_type="cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            ampscaler.scale(loss).backward()
            # ampscaler.unscale_(optimizer)
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            ampscaler.step(optimizer)
            ampscaler.update()
        else:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        outputs_dn = (outputs * (max_val - min_val)) + min_val
        targets_dn = (targets * (max_val - min_val)) + min_val
        r2_metric.update(outputs_dn.float(), targets_dn.float())
        total_loss += loss.float().item()
    r2 = r2_metric.compute().cpu().item()
    r2_metric.reset()
    return total_loss / len(loader), r2

In [19]:
def val_epoch(model, loader, criterion, device, min_val, max_val):
    model.eval()
    total_loss = 0.0
    r2_metric = R2Score().to(device)
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            with torch.amp.autocast(device_type=device.type):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            outputs_dn = (outputs * (max_val - min_val)) + min_val
            targets_dn = (targets * (max_val - min_val)) + min_val
            r2_metric.update(outputs, targets)
            total_loss += loss.float().item()
    r2 = r2_metric.compute().cpu().item()
    r2_metric.reset()
    return total_loss / len(loader), r2

In [ ]:
best_val_loss = float('inf')
train_val_logs = []
early_stop = False
epochs_no_improve = 0
early_stop_patience = 10
min_tgt_val, max_tgt_val = get_scaler_min_max(target_col, scaler_path, device)
print(f"Training {model_name} on device: {device}")

# Training Loop
for epoch in range(max_epochs):
    if early_stop:
        print(f"Early stopping at epoch {epoch}")
        break

    start_time = time.time()
    train_loss, train_r2 = train_epoch(model, train_loader, optimizer, 
                                       criterion, device, min_tgt_val, max_tgt_val, 
                                       ampscaler=ampscaler)
    val_loss, val_r2 = val_epoch(model, val_loader, criterion, device, min_tgt_val, max_tgt_val)
    scheduler.step(val_loss)

    # Early stop and save logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), f"models/notebook/{target_col}_{model_name}_v3.pth")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve > early_stop_patience:
            early_stop = True
    
    elapsed = time.time() - start_time
    curr_lr = optimizer.param_groups[0]['lr']
    train_val_logs.append({
        "epoch": epoch,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_r2": train_r2,
        "val_r2": val_r2,
        "learning_rate": curr_lr
    })
    print(f"Epoch: {epoch}/{max_epochs-1} | " 
          f"Train Loss: {train_loss:.5f}, Train R²: {train_r2:.4f} | "
          f"Val Loss: {val_loss:.5f}, Val R²: {val_r2:.4f} | "
          f"LR: {curr_lr}, Time: {round(elapsed, 2)} secs")

train_val_df = pd.DataFrame(train_val_logs)
train_val_df.to_csv(f"models/notebook/{target_col}_{model_name}_train_val_logs_v3.csv", index=False)

# Load best model weights before returning
model.load_state_dict(torch.load(f"models/notebook/{target_col}_{model_name}_v3.pth", weights_only=True))

Training MLP on device: cuda
Epoch: 0/9 | Train Loss: 0.01124, Train R²: 0.7503 | Val Loss: 0.00736, Val R²: 0.8410 | LR: 0.005, Time: 39.36 secs
Epoch: 1/9 | Train Loss: 0.00877, Train R²: 0.8051 | Val Loss: 0.00716, Val R²: 0.8452 | LR: 0.005, Time: 38.75 secs
Epoch: 2/9 | Train Loss: 0.00838, Train R²: 0.8139 | Val Loss: 0.00687, Val R²: 0.8516 | LR: 0.005, Time: 38.93 secs
Epoch: 3/9 | Train Loss: 0.00814, Train R²: 0.8193 | Val Loss: 0.00685, Val R²: 0.8520 | LR: 0.005, Time: 41.64 secs
Epoch: 4/9 | Train Loss: 0.00794, Train R²: 0.8237 | Val Loss: 0.00669, Val R²: 0.8555 | LR: 0.005, Time: 43.22 secs


In [ ]:
df_log = pd.read_csv(f"models/notebook/{target_col}_{model_name}_train_val_logs_v3.csv")
plt.figure(figsize=(8, 5))
sns.lineplot(x="epoch", y="train_loss", data=df_log, label="Train Loss")
sns.lineplot(x="epoch", y="val_loss", data=df_log, linestyle="--", label="Val Loss")
plt.title(f'Training and Validation Loss (Strategy-3) {model_name}, {target_col}')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()
# plt.savefig(f'{target_col}_yb_loss_curve.png', dpi=100, bbox_inches='tight')
plt.close()

In [18]:
def get_scenario(tdata, scenario):
    return tdata[
        (tdata['Treatment'] == scenario['Treatment']) &
        (tdata['NFirstApp'] == scenario['NFirstApp']) &
        (tdata['PlantingDay'] == scenario['PlantingDay']) &
        (tdata['Year'] == scenario['Year']) &
        (tdata['IrrgDep'] == scenario['IrrgDep']) &
        (tdata['IrrgThresh'] == scenario['IrrgThresh']) 
    ].sort_values("DayAfterPlant").reset_index(drop=True)


def evaluate_and_plot(df, model_name, model, scenario,
                     feature_cols, target_col, scaler_path, seq_len=None):
    sc_df = get_scenario(df, scenario)
    original_dap = sc_df['DayAfterPlant'].values
    sc_df =  preserve_original_columns(sc_df)
    sc_df = cumulate_columns(sc_df, "NApp")
    sc_df_norm, _ = mdl.normalize_columns(
        sc_df, 
        feature_cols + [target_col],
        mode="transform", 
        scaler_path=scaler_path
    )
    X_tensor, _ = process_data(sc_df_norm, feature_cols, target_col, seq_len=seq_len)
    y_true = sc_df[target_col].values
    
    model.eval()
    with torch.no_grad():
        y_pred = model(X_tensor).cpu().numpy().flatten()
        
    # Step 6: De-normalize predictions
    pred_df = pd.DataFrame({
        target_col: y_pred,
        'DayAfterPlant': original_dap[seq_len-1:] if seq_len != None else original_dap
    })
    pred_df = mdl.denormalize_columns(
        pred_df, 
        [target_col], 
        scaler_path=scaler_path
    )
    y_pred = pred_df[target_col].values
    
    fig, ax1 = plt.subplots(figsize=(8, 5))
    
    ax1.plot(original_dap, y_true, label=f'True {target_col}', color='blue')
    if seq_len is None:
        ax1.plot(original_dap, y_pred, label=f'Predicted {target_col}', color='red')
    else:
        ax1.plot(original_dap[seq_len-1:], y_pred, label=f'Predicted {target_col}', color='red')
    ax1.set_xlabel('DayAfterPlant')
    ax1.set_ylabel(target_col)
    # ax1.set_ylim(0, 100)
    
    ax2 = ax1.twinx()
    ax2.plot(original_dap, sc_df['Rain'], label='Rain', color='lightgreen', linestyle='--', alpha=0.5)
    ax2.set_ylabel('Rain (mm)', color='lightgreen')
    ax2.set_ylim(0, 50)
    ax2.invert_yaxis()
    
#     # Add N application markers
    n_first_app = scenario['NFirstApp']
    if n_first_app == "Npre":
        app_daps = [-36, 22, 44]  # Pre-plant, emergence, tillering
    elif n_first_app == "Npl":
        app_daps = [0, 22, 44]    # Planting, emergence, tillering
    else:
        app_daps = []
    for dap in app_daps:
        if dap in original_dap:  # Check if DAP exists in scenario data
            ax1.axvline(x=dap, color='gray', linestyle='--', linewidth=2.0, alpha=0.8)
            ax1.text(dap, ax1.get_ylim()[1], f'N\n↓', 
                     ha='center', va='bottom', color='gray')
    
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    plt.title(f'Scenario: {scenario}')
    plt.tight_layout()
    plt.show()

In [19]:
# model_name = "LSTM"
# model = mdl.LSTM(input_dim=len(feature_cols), 
#                  num_layers=2, 
#                  hidden_size=128)
# model_name = "EncoderOnlyTransformer"
# model = mdl.EncoderOnlyTransformer(
#         input_dim=len(feature_cols), num_layers=2,
#         d_model=128, max_seq_len=seq_len)
# model_name = "MLP"
# model = mdl.MLP(input_dim=len(feature_cols), 
#                        hidden_size=256, 
#                        num_layers=2, 
#                        dropout=0.2)
model_name="Linear_Regression"
model = mdl.LinearRegression(input_dim=len(feature_cols))
model.load_state_dict(torch.load(f"models/notebook/{target_col}_{model_name}_v3.pth", weights_only=True))
# model=mdl.MLP(input_dim=len(feature_cols), hidden_size=256, num_layers=4)
# model = mdl.TCN(input_dim=len(feature_cols), num_channels=[256, 128, 64, 32, 16, 8], kernel_size=5)

print(f"Training {model_name} on device: {device}")
print(summary(model))
scenario = {
    'Treatment': '0-112-112',
    'NFirstApp': 'Npre',
    'PlantingDay': 15,
    'Year': 2020,
    'IrrgDep': 50,
    'IrrgThresh': 70
}
# get_scenario(test_data, scenario)
evaluate_and_plot(
    df, model_name, model, 
    scenario, feature_cols,target_col,
    scaler_path=f"models/notebook/{target_col}_scaler_v1.pkl",
    seq_len=seq_len
)

FileNotFoundError: [Errno 2] No such file or directory: 'models/notebook/NTotL1_Linear_Regression_v3.pth'

In [16]:
def evaluate_model(model_name, model, X_tensor, y_tensor, feature_cols, 
                   target_col, batch_size, device, 
                   scaler_path, seq_len=None):
    model.to(device)
    model.eval()
    all_preds = []
    all_targets = []

    # X_tensor, y_tensor = process_data(df, feature_cols, target_col, 
    #                             scaler_path=scaler_path, mode="transform", seq_len=seq_len)
    tensor_dataset = TensorDataset(X_tensor, y_tensor)
    data_loader = DataLoader(tensor_dataset, batch_size, shuffle=False)
    with torch.inference_mode():
        for i, (inputs, targets) in enumerate(data_loader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            
            # Flatten and format
            targets = targets.numpy().flatten()
            outputs = outputs.cpu().numpy().flatten()

            outputs_denorm = inverse_normalize(outputs, target_col, scaler_path=scaler_path)
            targets_denorm = inverse_normalize(targets, target_col, scaler_path=scaler_path)
            
            # print(targets[:10], outputs[:10])
            # print(targets_denorm[:10], outputs_denorm[:10])
            # return
            all_preds.append(outputs_denorm)
            all_targets.append(targets_denorm)
            pgrs = (i+1) * 20 // len(data_loader)
            print(f"{i+1}/{len(data_loader)}[{'=' * pgrs}>{' ' * (19 - pgrs)}]", end='\r')
    # Concatenate all predictions and targets
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_targets)
    return compute_metrics(y_true, y_pred)

In [32]:
result_table = {}
result_table['Strategy_3'] = {}

In [17]:
# Load trained MLP model
# model_name = "EncoderOnlyTransformer"
# model = mdl.EncoderOnlyTransformer(
#         input_dim=len(feature_cols), num_layers=2,
#         d_model=128, max_seq_len=seq_len)
# model_name = "MLP"
# model = mdl.MLP(input_dim=len(feature_cols), 
#                        hidden_size=256, 
#                        num_layers=2, 
#                        dropout=0.2)
model_name="LinearRegression"
model = mdl.LinearRegression(input_dim=len(feature_cols))
model.load_state_dict(torch.load(f"models/{target_col}/{target_col}_{model_name}.pth", weights_only=True))

# result_table['Strategy_3'][target_col] = {}
mae, rmse, r2 = evaluate_model(
    model_name, model, X_val, y_val, 
    feature_cols, target_col,b_sz, device,
    scaler_path=f"models/{target_col}/{target_col}_scaler.pkl", 
    seq_len=seq_len
)
# result_table['Strategy_3'][target_col]["train"] = {
#     "mae": mae,
#     "rmse": rmse,
#     "r2": r2
# }

# mae, rmse, r2 = evaluate_model(
#     model_name, model, val_split, 
#     feature_cols, target_col,b_sz, device,
#     scaler_path=f"models/notebook/{target_col}_scaler_v3.pkl", 
#     seq_len=seq_len
# )
# result_table['Strategy_3'][target_col]["val"] = {
#     "mae": mae,
#     "rmse": rmse,
#     "r2": r2
# }


# mae, rmse, r2 = evaluate_model(
#     model_name, model, test_split, 
#     feature_cols, target_col,b_sz, device,
#     scaler_path=f"models/notebook/{target_col}_scaler_v3.pkl", 
#     seq_len=seq_len
# )
# result_table['Strategy_3'][target_col]["test"] = {
#     "mae": mae,
#     "rmse": rmse,
#     "r2": r2
# }


17.296642 21.999334 0.482450008392334


In [34]:
result_table

{'Strategy_3': {'NTotL1': {'train': {'mae': np.float32(17.390928),
    'rmse': np.float32(22.136848),
    'r2': 0.49526357650756836},
   'val': {'mae': np.float32(15.866853),
    'rmse': np.float32(21.25926),
    'r2': 0.3508443832397461},
   'test': {'mae': np.float32(14.44062),
    'rmse': np.float32(19.055813),
    'r2': 0.39113956689834595}}}}

In [35]:
flattened_data = []
for strategy, strategy_data in result_table.items():
    for target, target_data in strategy_data.items():
        for metric_type, metrics in target_data.items():
            row = {
                "Strategy": strategy,
                "Target": target,
                "Metric_Type": metric_type
            }
            row.update(metrics)
            flattened_data.append(row)
metric_df = pd.DataFrame(flattened_data)
metric_df = metric_df.set_index(['Strategy', 'Target', 'Metric_Type'])
metric_df = metric_df.sort_values(by=['Strategy', 'Target'])
# metric_df.to_csv("models/notebook/metrics.csv", index=False)
metric_df

mae       rmse        r2
Strategy   Target Metric_Type                                
Strategy_3 NTotL1 train        17.390928  22.136848  0.495264
                  val          15.866853  21.259260  0.350844
                  test         14.440620  19.055813  0.391140

In [41]:
# def evaluate_by_scenario(model, df, feature_cols, target_col, scaler_path, seq_len=None):
#     model.eval()
#     results = []
#     scenario_vars = ['Year', 'PlantingDay', 'Treatment', 'OrgIrrgDep', 'OrgIrrgThresh', 'NFirstApp']
#     for scenario_name, group in df.groupby(scenario_vars, observed=True):
#         X, y_true = process_data(group, feature_cols, target_col=target_col, seq_len=seq_len)
#         X_tensor = torch.FloatTensor(X).to(device)
#         with torch.no_grad():
#             y_pred = model(X_tensor).cpu().numpy().flatten()
        
#         y_true = y_true.cpu().numpy().flatten()
#         # Denormalize before calculating metrics
#         y_pred = inverse_normalize(y_pred, target_col, scaler_path=scaler_path)
#         y_true = inverse_normalize(y_true, target_col, scaler_path=scaler_path)
        
#         metrics = compute_metrics(y_true, y_pred)
#         results.append({
#             "Scenario": "_".join([str(s) for s in scenario_name]),
#             "MAE": mae,
#             "RMSE": rmse,
#             "R²": r2
#         })
#     results_df = pd.DataFrame(results)
#     avg_mae = results_df["MAE"].mean()
#     avg_rmse = results_df["RMSE"].mean()
#     avg_r2 = results_df["R²"].mean()
#     return results_df, avg_mae, avg_rmse, avg_r2


# def plot_r2_histogram(df, data_type):
#     print("")
#     plt.figure(figsize=(6, 4))
#     ax = sns.histplot(df["R²"], bins=40)
#     plt.title(f"{data_type.capitalize()}, Distribution of Scenario-Level R² Scores")
#     plt.xlabel("R²")
#     plt.ylabel("Frequency")
#     for p in ax.patches:
#         height = p.get_height()
#         if height > 0:  # Only annotate bars with height > 0
#             coords = (p.get_x() + p.get_width() / 2.0, height)
#             ax.annotate(f'{int(height)}', coords, 
#                         ha='center', va='bottom', fontsize=8)
#     plt.legend([f'Total samples: {len(df)}'], loc='upper left')
#     plt.show()

In [ ]:
# [512, 10, 5] -> [512, 1]
# to:
# [512, 150, 5] -> [512, 150]
# [X] TCN, LSTM, Trasnformers
# R2, MAE, RMSE -> scenario_wise



# LR, XgBoost, mlp
# [512, 5] -> [512,1]
# [1, 5] -> [1, 1]
# R2, MAE, RMSE
# test_df -> 1 at a time [108,5] -> [108,1] mean(scenario_results)